
# Utility


### Imports and Constants


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import time
import math
import cv2
from cv2 import dnn
import os
import PIL
from IPython.display import display
from ipywidgets import Image
import keras
import numpy as np
import glob
import tqdm
import tensorflow as tf
from sklearn.preprocessing import normalize
import pickle
import tensorflow as tf
import os
import random
from tensorflow.python.keras.models import Sequential
from tensorflow.keras import layers
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np


BASE_DIR = ''
GDRIVE_DIR = "gdrive/MyDrive/Project/"
ART_DATASET = BASE_DIR + "ArtImages/dataset_updated/"
TRAINING_SET = ART_DATASET + "training_set/"
VALIDATION_SET = ART_DATASET + "validation_set/"
DISTRACTOR_SET = BASE_DIR + "mirflickr/"
QUERY_SET = "query"
IMAGE_SIZE = (229,229)
BATCH_SIZE = 32
BUCKET_SIZE = 300

###Utility Functions

In [ ]:
def display_image(filename, score=0):
    
    filepath = os.path.join(GDRIVE_DIR , filename)
    image = Image.from_file(filepath, width=300, height=400)
    print('{} - {:.3f}'.format(filename, score))  # :.3f = 3-decimal float
    display(image)

# Function to create the features schema, receives the dnn extractor, the dataset, 
# the features_list to which append the features. 
def create_feature_schema(dnn, features_list, dataset, distractor=False, remove_classifier=False,layer_to_stop='gap'):
    # extract features from the given dataset
    features = dnn.extract_features(dataset, normalization=True, remove_classifier=remove_classifier,layer_to_stop=layer_to_stop)
    indx = 0
  
    for i in range(0,len(dataset.filenames)):
        print(i)
        if distractor == True:
          label="Distractor"
        else:
          label = dataset.filenames[i].split("/")[0]
        #create the feature schema and append it
        feature_schema = FeaturesSchema(tensor = features[i], img_id=dataset.filenames[i], label=label)
        features_list.append(feature_schema)

# function to read training, validation and distractor using flow from directory.
# it's used to perform data augmentation for training 
def read_data(shuffle=True):
    train_datagen = image.ImageDataGenerator(
            preprocessing_function=preprocess_input,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=20,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=0.2)

    test_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)

    validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)
    # we split training in both training and validation since we don't have 
    # a test set
    train_generator = train_datagen.flow_from_directory(
            TRAINING_SET,
            seed=123,
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='sparse',
            shuffle=shuffle,
            subset='training')

    validation_generator = validation_datagen.flow_from_directory(
            TRAINING_SET,
            seed=123,
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='sparse',
            shuffle=shuffle,
            subset='validation')
    
    distractor_dataset = test_datagen.flow_from_directory(
        DISTRACTOR_SET,
        shuffle=shuffle,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE)

    return validation_generator, train_generator, distractor_dataset


# function that reads training, validation and distractor using flow from directory
# it's used to create the feature schemas during extraction
def read_data_correct():
    datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)
    train_generator = datagen.flow_from_directory(
            TRAINING_SET,
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='sparse',
            shuffle=False)
    test_generator = datagen.flow_from_directory(
            VALIDATION_SET,
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='sparse',
            shuffle=False)
    distractor_dataset = datagen.flow_from_directory(
        DISTRACTOR_SET,
        shuffle=False,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE)
    return test_generator, train_generator, distractor_dataset

# function to remove the corrupted images from the dataset  
def remove_corrupted_images(path):
  ids = glob.glob(path + "*.jpg")
  for img in ids:
    try:
      image.load_img(img, target_size=(150,150))
    except:
      print(img, "removed!")
      os.remove(img)

# function to count the number of elements for each class inside the dataset.
# labels is a list containing all the labels of the elements
def count_label(labels):
  print(len(labels))
  drawings = 0
  engraving = 0
  iconography = 0
  painting = 0
  sculpture = 0
  for label in labels:
    label = list(label)
    drawings += label.count(0)
    engraving += label.count(1)
    iconography += label.count(2)
    painting += label.count(3)
    sculpture += label.count(4)
  return drawings,engraving,iconography,painting, sculpture

# function to get the model with the added layers
def get_model():
    pretrained_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(229,229,3))
    model = Sequential()
    model.add(pretrained_model)
    model.add(layers.GlobalAveragePooling2D(name='gap'))
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(512, activation='relu', name="last_relu"))
    model.add(tf.keras.layers.Dropout(0.3, name="last_dropout"))
    model.add(tf.keras.layers.Dense(5, activation='softmax',name='classifier_hidden'))
    model.summary()
    return model

def copy_zip_to_colab():
  !rm -r "ArtImages"
  !rm -r "mirflickr"

  zip_path = '/content/gdrive/MyDrive/Project/ArtImages.zip'
  !cp "{zip_path}" .
  !unzip -q "/content/ArtImages.zip"
  !rm "/content/ArtImages.zip"

  zip_path = '/content/gdrive/MyDrive/Project/mirflickr25k.zip'
  !mkdir "/content/mirflickr"
  !cp "{zip_path}" "/content/mirflickr"
  !unzip -q "/content/mirflickr/mirflickr25k.zip" -d "/content/mirflickr/"
  !rm "/content/mirflickr/mirflickr25k.zip"

# function to remove images that are both in training and validation from the validation
def remove_duplicates():
  with open(GDRIVE_DIR+"features_extracted_not_fine_tuned", 'rb') as features_file:
    feature_schemas = pickle.load(features_file)
  vptree = VPTree(feature_schemas[0:7721-1], BUCKET_SIZE) 
  count = 0
  for image in feature_schemas[7721:7721+856-1]:
    results = vptree.knn(image, 1)
    for result in results:
      if result[0] <= 0.1:
        print("query image: ",image.img_id)
        print(result[0])
        print(result[1].img_id)
        print(result[1].label)
        os.remove(GDRIVE_DIR+"ArtImages/dataset_updated/validation_set/"+image.img_id)
  print("Count: ",count)


In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


# Feature Schema
This class is used to represent features extracted by the network.


*   Tensor: it will contain the features extracted by the nn
*   Image id: it will contain the filename of the image
*   Label: it will contain the class (Drawings, engraving, iconography, painting, sculpture) of the image


In [ ]:
class FeaturesSchema:
  def __init__(self, tensor, img_id, label):
    self.tensor = tensor
    self.img_id = img_id
    self.label = label

  # function used to evaluate the cosine similarity between the features of the current object 
  # and the features of the schema parameter (it's a FeatureSchema object)
  def distance_from(self, schema): 
    return 1 - np.dot(np.array(self.tensor), np.array(schema.tensor))/np.linalg.norm(np.array(self.tensor))

# VPTree
This class will be used to index features extracted. But first we need to explain some other classes.<br>


###Priority Queue
This class will be used to optimize the results of the knn search on the index.


*   Data: it will contain the features vector and the priority of them
*   Index: variable to check the number of elements contained in the queue
*   Size: variable that contains the max size of the queue


In [ ]:
import heapq
class PriorityQueue:
  def __init__(self,k):
    self.data = []
    self.index = 0
    self.size = k

  # Function to add element to a queue, according to its priority
  def push(self, priority, item):
    heapq.heappush(self.data,(-priority,self.index,item))
    self.index += 1

  # Function to determine if the queueu is full or not
  def is_full(self):
    return len(self.data) == self.size

  # Function to remove an element from the queue
  def pop(self):
    return heapq.heappop(self.data)

###Node
This class will represent a node inside the VPTree. This can be a leaf or a pivot node.

*   Leaf: if true, the node is a leaf. It's a pivot otherwise.
*   Left, Right: if the node is a pivot, they will point to the left and right sons.
*   Median: if the node is a pivot, it will contain the median value.
*   Pivot: if the node is a pivot, it will contain the features of the pivot.
*   Subset: if the node is a leaf, it will contain all the features schemas contained in it.


In [ ]:
class Node:
  def __init__(self):
    self.leaf = False
  
  def add_left(self, node):
    self.left = node
  
  def add_right(self, node):
    self.right = node

  def set_median(self, median):
    self.median = median

  def set_pivot(self, pivot):
    self.pivot = pivot

  def get_median(self):
    return self.median
  
  def get_pivot(self):
    return self.pivot

  def get_left(self):
    return self.left
  
  def get_right(self):
    return self.right

  def set_leaf(self, subset):
    self.subset = subset
    self.leaf = True

  def get_subset(self):
    return self.subset

  def is_leaf(self):
    return self.leaf

###VPTree


*    bucket_size: maximum size of elements contained in a leaf node.
*    feature_schemas = array that contains the features schemas of all the elements (used to build the tree and to do performance comparisons).
*    nodes: number of nodes contained in the tree.
*    root = pointer to the root node.

In [ ]:
class VPTree:

  # Recursive function used to build the tree
  def build(self, node, feature_subset, leaf=False):
    
    if leaf is True:
      # if the node is a leaf we have to enter the features schemas in its bucket
      node.set_leaf(feature_subset)
    else:
      # if the node is an internal node, we randomly choose the pivot      
      pivot = random.choice(feature_subset)
      # then we evaluate the distances between the pivot and all the other objects
      distances = list()
      for feature_schema in feature_subset:
        distances.append(feature_schema.distance_from(pivot))
      distances = np.array(distances)
      # evaluate the median distance
      median = np.median(distances)
      subset1 = list()
      subset2 = list()
      for feature_schema in feature_subset:
        # if the distance is lower than the media, the feature_schema will go in the left subset (subset1)
        if feature_schema.distance_from(pivot) <= median:
          subset1.append(feature_schema)
        else:
        # otherwise it will go in the right subset
          subset2.append(feature_schema)
      # setting the values for the pivot node
      node.set_median(median)
      node.set_pivot(pivot)
      # add left and right sons, increment the number of nodes 
      node.add_left(Node())
      node.add_right(Node())
      self.nodes += 2
      # if the length of the left subset is lower than the bucket size, then we can directly build the leaf node
      if len(subset1) <= self.bucket_size:
        self.build(node.get_left(), subset1, leaf=True)
      else:
      # otherwise we have to do all the above procedure for another pivot, splitting the data again...
        self.build(node.get_left(), subset1)
      # same for right subset
      if len(subset2) <= self.bucket_size:
        self.build(node.get_right(), subset2, leaf=True)
      else:
        self.build(node.get_right(), subset2)
  
  def __init__(self, feature_schemas, bucket_size):
    self.bucket_size = bucket_size
    self.feature_schemas = feature_schemas
    self.nodes = 1
    self.root = Node()
    self.build(self.root, feature_schemas)


  # function to print the tree, it will call print_tree
  def print_root(self):
    self.print_tree(self.root)

  # recursive function to print the tree
  def print_tree(self, node):
    if node.is_leaf():
      print("LEAF")
      print(len(node.subset))
      return
    else:
      print("INTERNAL")
      print("Pivot: " , node.get_pivot().tensor)
      print("Median: " , node.get_median())
      self.print_tree(node.get_left())
      self.print_tree(node.get_right())
      return

  # function to perform the range search for a given query and a certain range
  def range_search(self, query, range):
    self.range_search_return_list = list()
    self.recursive_range_search(self.root, query, range)
    return self.range_search_return_list
    
  # recursive function to perform the range search
  def recursive_range_search(self, node, query, range):
    if node.is_leaf() == True:
      # if the node is a leaf, then we have to search in its bucket for objects
      # that have a smaller distance than the range. These objects will be 
      # inserted in the result
      for object_schema in node.get_subset():
        if query.distance_from(object_schema) <= range:
          self.range_search_return_list.append(object_schema)
      return
    # if the distance between the query and the pivot is smaller than the 
    # range, we insert the pivot in the result
    if query.distance_from(node.get_pivot()) <= range:
      self.range_search_return_list.append(node.get_pivot())
    # if the distance between the query and the pivot minus the range is smaller
    # than the median, we have to search in the left son of the pivot
    if query.distance_from(node.get_pivot()) - range <= node.get_median():
      self.recursive_range_search(node.get_left(), query, range)
    # otherwise we have to search in the right son
    if query.distance_from(node.get_pivot()) + range >= node.get_median():
      self.recursive_range_search(node.get_right(), query, range)
    return

  # function to perform the knn search for a given query a certain k
  def knn(self, query, k):
    # contains the maximum distance between the query and the neighbors 
    # in the result
    self.d_max = math.inf
    # priority queue to save the results of the knn
    self.nn = PriorityQueue(k)
    # perform the knn search on the tree
    self.recursive_knn(self.root, query, k)
    results = list()
    # once the knn is finished, we extract the results from
    # the priority queue and we return them
    while k != 0:
      tmp = self.nn.pop()
      results.append((abs(tmp[0]), tmp[2]))
      k -= 1
    return list(reversed(results))
  
  # recursive function to perform the knn search
  def recursive_knn(self, node, query, k):
    if node.is_leaf() == True:
      # if the node is a leaf, it's the same reasoning applied to a pivot,
      # with the exception that we have to repeat this reasoning for all
      # the objects in the leaf's bucket.
      for object_schema in node.get_subset():
        distance = query.distance_from(object_schema)
        if not self.nn.is_full():
            self.nn.push(priority = distance, item = object_schema)
            self.d_max = distance if self.d_max < distance else self.d_max
        elif abs(distance) < abs(self.d_max):
            self.nn.pop()
            self.nn.push(priority = distance, item = object_schema)
            tmp = self.nn.data[0][0]
            self.d_max = abs(tmp)
      return
    # if the node is a pivot, we check the distance from the query and the pivot
    distance = query.distance_from(node.get_pivot())
    if not self.nn.is_full():
        # if the priority queue is not full, we insert the pivot in it and
        # check if the distance between the query and the pivot is greater than
        # the current d_max. If this is true, we update d_max to this distance.
        # Otherwise, d_max is not updated.
        self.nn.push(priority = distance, item = node.get_pivot())
        self.d_max = distance if self.d_max < distance else self.d_max
    elif abs(distance) < abs(self.d_max):
        # if the priority queue is full, we have to check if the distance 
        # between the pivot and the query is lower than the d_max. If this
        # happens, we remove the object with the highest distance
        # and we insert the pivot in the priority queue. In addition, we
        # update d_max to the distance between the current pivot and the query
        self.nn.pop()
        self.nn.push(priority = distance, item = node.get_pivot())
        tmp = self.nn.data[0][0]
        self.d_max = abs(tmp)
    # if the distance between the pivot and the query minus the current max 
    # distance is lower or equal than the median of the pivot, we have to search 
    # in the left son.
    if distance - self.d_max <= node.get_median():
      self.recursive_knn(node.get_left(), query, k)
    # otherwise we search in the right son.
    if distance + self.d_max >= node.get_median():
      self.recursive_knn(node.get_right(), query, k)
    return

  # function that performs the knn search in a linear way, without using the
  # index. Used for performance comparisons. 
  def knn_not_optimized(self, query, k):
    results = list()
    for object_schema in self.feature_schemas:
      distance = object_schema.distance_from(query)
      results.append((distance, object_schema))
    print(len(self.feature_schemas))
    return sorted(results, key=lambda x: x[0])

# DNNExtractor
This class is used perform features extraction.


* net: the network used to extract features.
* batch_size: size of the batch.
* trainable: if true, the network can be fine-tuned.

In [ ]:
class DNNExtractor:    
    
    def __init__(self, model, batch_size, trainable):
      model.trainable = trainable
      self.net = model
      self.batch_size = batch_size
      self.trainable = trainable

    def extract_features_from_single_image(self, filename, remove_classifier=True,layer_to_stop="gap"):
        data_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
        query_gen = data_gen.flow_from_directory(
            QUERY_SET,
            shuffle=False,
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE
        )
        query_features = self.extract_features(query_gen, True, remove_classifier=remove_classifier,layer_to_stop=layer_to_stop)
        query_feature_schema = FeaturesSchema(query_features, filename, "")
        return query_feature_schema

    # function used to extract features from the given dataset. 
    def extract_features(self, dataset, normalization=False, remove_classifier=False,layer_to_stop='gap'):
      if remove_classifier:
          # if remove_classifier is true, we remove the last layer. 
          # remove_classifier = False when using pre-trained network 
          self.net = Model(inputs=self.net.input, outputs=self.net.get_layer(layer_to_stop).output)
      dataset_extracted = self.net.predict(dataset, batch_size=self.batch_size)
      if normalization is True:
        dataset_extracted = normalize(dataset_extracted)
      return dataset_extracted

    # function to perform the fine-tuning of the network with the given parameters.
    def fine_tune(self, training_set, validation_set, optimizer, loss, metric, learning_rate, epochs, class_weight):
      if not self.trainable:
        print("Net not trainable")
        return
      if optimizer == "Adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate)  
      elif optimizer == "RMSprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate)

      self.net.compile(optimizer = optimizer, loss = loss, metrics = [metric])

      # early stopping
      es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
      mc = ModelCheckpoint('best_model', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
      # fit model
      return self.net.fit(training_set, validation_data = validation_set, epochs=epochs, steps_per_epoch=np.ceil(6179/float(BATCH_SIZE)), 
                   validation_steps=np.ceil(1542/float(BATCH_SIZE)), class_weight=class_weight, callbacks=[es, mc])


# Pre-Trained Network
In this section we are going to use the pre-trained network to extract features from the images.

In [ ]:
%ls mirflickr2

ls: cannot access 'mirflickr2': No such file or directory


In [ ]:
copy_zip_to_colab()

rm: cannot remove 'ArtImages': No such file or directory
rm: cannot remove 'mirflickr': No such file or directory


In [ ]:
test_generator, train_generator, distractor_dataset  = read_data_correct()

#count labels 
labels =list()
batch_index = 0
while batch_index <= test_generator.batch_index:  
    data = test_generator.next()
    labels.append(data[1])
    batch_index = batch_index + 1
drawings,engraving,iconography,painting, sculpture = count_label(labels)
print("Drawings: ",drawings)
print("Engravings: ",engraving)
print("Iconography: ",iconography)
print("Painting: ",painting)
print("Sculpture: ",sculpture)

Found 7721 images belonging to 5 classes.
Found 603 images belonging to 5 classes.
Found 25000 images belonging to 1 classes.
19
Drawings:  52
Engravings:  24
Iconography:  200
Painting:  206
Sculpture:  121


In [ ]:
# get the pretrained model and add to it a global average pooling layer
pretrained_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(229,229,3))
model = Sequential()
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
model.summary()
# build a DNNExtractor
dnn = DNNExtractor(model, BATCH_SIZE, False)
# read data for extracting features
test_generator, train_generator, distractor_dataset = read_data_correct()
# creating the features
feature_schemas = list()
create_feature_schema(dnn, feature_schemas, train_generator, distractor=False, remove_classifier=False)
create_feature_schema(dnn, feature_schemas, test_generator, distractor=False, remove_classifier=False)
create_feature_schema(dnn, feature_schemas, distractor_dataset, distractor=True, remove_classifier=False)
# save features extracted into a file
with open(GDRIVE_DIR + 'features_extracted_not_fine_tuned', 'wb') as features_file:
  pickle.dump(feature_schemas, features_file)

87916544/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________
Found 7721 images belonging to 5 classes.
Found 603 images belonging to 5 classes.
Found 25000 images belonging to 1 classes.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Streaming output truncated to the last 5000 lines.
20000
20001
20002
20003
20004
20005
20006
20007
20008
20009
20010
20011
20012
20013
20014
20015
20016
20017
20018
20019
20020
20021
20022
20023
20024
20025
20026
20027
20028
20029
20030
20031
20032
20033
20034
20035
20036
20037
20038
20039
20040
20041
20042
20043
20044
20045
20046
20047
20048
20049
20050
20051
20052
20053
20054
20055
20056
20057
20058
20059
20060
20061
20062
20063
20064
20065
20066
20067
20068
20069
20070
20071
20072
20073
20074
20075
20076
20077
20078
20079
20080
20081
20082
20083
20084
20085
20086
20087
20088
20089
20090
20091
20092
20093
20094
20095
20096
20097
20098
20099
20100
20101
20102
20103
20104
20105
20106
20107
20108
20109
20110
20111
20112
20113
20114
20115
20116
20117
20118
20119
20120
20121
20122
20123
20124
20125
20126
20127
20128
20129
20130
20131
20132
20133
20134
20135
20136
20137
20138
20139
20140
20141
20142
20143
20144
20145
20146
20147
20148
20149
20150
20151
20152
20153
20154
20155
20156
20157
2

# Fine-Tuned Network

###Fine-Tuning

In [ ]:

# DONE OFFLINE USING GPU
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
# CHECK IF GPU IS RUNNING
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Create the model
validation_generator, train_generator, _ = read_data()
model = get_model()

fine_tuned_dnn = DNNExtractor(model, BATCH_SIZE, True)
#count labels 
labels =list()
batch_index = 0
while batch_index <= train_generator.batch_index:
    data = train_generator.next()
    labels.append(data[1])
    batch_index = batch_index + 1
drawings,engraving,iconography,painting, sculpture = count_label(labels)
total = drawings + engraving + iconography + painting + sculpture
# set parameter for fine tuning
EPOCHS_NUMBER = 100
LEARNING_RATE = 0.0001
OPTIMIZER = "Adam"
LOSS_FUNCTION = "sparse_categorical_crossentropy"
METRIC = "accuracy"
class_weight = {0: (total-drawings)/total, 1: (total-engraving)/total, 2: (total-iconography)/total, 3: (total-painting)/total, 4: (total-sculpture)/total}
# fine tune the network
history = fine_tuned_dnn.fine_tune(train_generator, validation_generator, 
                        OPTIMIZER, LOSS_FUNCTION, METRIC, LEARNING_RATE, EPOCHS_NUMBER,class_weight)
# save the model
fine_tuned_dnn.net.save(os.path.join(GDRIVE_DIR,"fine_tuned_model"))

2.4.0


SystemError: ignored

In [ ]:
# see how the accuracy and the loss change during training
mport matplotlib.pyplot as plt
plt.title('ACCURACY')
plt.plot(history.history['accuracy'],'bo-',label = 'Training Accuracy')
plt.plot(history.history['val_accuracy'],'ro-',label = 'Validation Accuracy')
plt.legend()
plt.show()
plt.title('LOSS')
plt.plot(history.history['loss'],'bo-',label = 'Training Loss')
plt.plot(history.history['val_loss'],'ro-',label = 'Validation Loss')
plt.legend()
plt.show()

In [ ]:
# MODEL EVALUATION ON TEST
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

y_labels =list()
batch_index = 0
while batch_index <= test_generator.batch_index:
    data = test_generator.next()
    for tmp in data[1]:
      y_labels.append(tmp)
    batch_index = batch_index + 1

y_pred1 = fine_tuned_dnn.net.predict(test_generator)
y_pred = np.argmax(y_pred1, axis=1)
print("Accuracy: ",fine_tuned_dnn.net.evaluate(test_generator)[1])
print("Precision: ",precision_score(y_labels, y_pred , average="macro"))
print("Recall: ",recall_score(y_labels, y_pred , average="macro"))
print("F1 Score: ",f1_score(y_labels, y_pred , average="macro"))

###Features extraction

In [ ]:
# read data for extracting features
test_generator, train_generator, distractor_dataset = read_data_correct()
model = tf.keras.models.load_model(os.path.join(GDRIVE_DIR,"fine_tuned_model"))
model.summary()
fine_tuned_dnn = DNNExtractor(model, BATCH_SIZE, True)
print("model is loaded")

# extract features until dense layer
feature_schemas = list()
create_feature_schema(fine_tuned_dnn, feature_schemas, train_generator, distractor=False, remove_classifier=True,layer_to_stop="last_dropout")
create_feature_schema(fine_tuned_dnn, feature_schemas, test_generator, distractor=False, remove_classifier=True,layer_to_stop="last_dropout")
create_feature_schema(fine_tuned_dnn, feature_schemas, distractor_dataset, distractor=True, remove_classifier=True,layer_to_stop="last_dropout")
# save extracted features
with open(GDRIVE_DIR + 'features_extracted_until_dense_layer', 'wb') as features_file:
  pickle.dump(feature_schemas, features_file)

# Extract features until gap layer
feature_schemas = list()
create_feature_schema(fine_tuned_dnn, feature_schemas, train_generator, distractor=False, remove_classifier=True)
create_feature_schema(fine_tuned_dnn, feature_schemas, test_generator, distractor=False, remove_classifier=True)
create_feature_schema(fine_tuned_dnn, feature_schemas, distractor_dataset, distractor=True, remove_classifier=True)
# save extracted features
with open(GDRIVE_DIR + 'features_extracted_until_gap_layer', 'wb') as features_file:
  pickle.dump(feature_schemas, features_file)

# Performance

In [ ]:
def evaluate_knn_search(queries_features, query_labels, vptree, k):
  query_labels = query_labels.reshape(-1, 1)  # reshape to use numpy broadcasting
  topk_labels = list()
  topk_scores = list()
  topk_images = list()
  for query_features in queries_features:
      results = vptree.knn(query_features, k)
      tmp_k_scores = list()
      tmp_k_images = list()
      tmp_k_labels = list()
      for result in results:
          tmp_k_scores.append(result[0])
          tmp_k_images.append(result[1].img_id)
          tmp_k_labels.append(result[1].label)
      topk_labels.append(tmp_k_labels)
      topk_scores.append(tmp_k_scores)
      topk_images.append(tmp_k_images)

  topk_is_relevant = topk_labels == query_labels
  #average precision
  aps = [sklearn.metrics.average_precision_score(l, s) for l,s in zip(topk_is_relevant, topk_scores)]
  print('APs per Query:', aps)
  print('mAP:', np.mean(aps))
  return np.mean(aps)


with open(GDRIVE_DIR+"features_extracted_until_dense_layer", 'rb') as features_file:
  feature_schemas = pickle.load(features_file)

vptree = VPTree(feature_schemas, BUCKET_SIZE)
mAPs = list()
feature_schemas = feature_schemas[7721:7721+603-1]

# mAP
for i in range(0,10):
  
  test_images = list()
  test_labels = list()
  for feature_object in feature_schemas:
    test_images.append(feature_object.tensor)
    test_labels.append(feature_object.label)

  test_images = np.array(test_images)
  test_labels = np.array(test_labels)
  n_queries = 50
  query_indexes = np.random.choice(len(test_images), n_queries, replace=False)
  query_images = test_images[query_indexes]
  query_labels = test_labels[query_indexes]
  feature_schemas = np.array(feature_schemas)

  queries_features = feature_schemas[query_indexes]

  k = 10
  #for query_features in queries_features
  mAPs.append(evaluate_knn_search(queries_features, query_labels, vptree, k))

print("Average Precision:")
for mAP in mAPs:
  print(mAP)

In [ ]:
#TIME EVALUATION
import random 
import time
with open(GDRIVE_DIR+"features_extracted_until_gap_layer", 'rb') as features_file:
  feature_schemas = pickle.load(features_file)

MAX_LEVEL = 5
BUCKET_SIZE = 300

long_feature_schemas = list()
for i in range(0, 3):
  long_feature_schemas += feature_schemas
vptree = VPTree(long_feature_schemas, BUCKET_SIZE)


times_optimized = []
times_not_optimized = []
k=10

for i in range(0,10):  
  print("Length: ",len(long_feature_schemas))
  query_index = random.randrange(len(long_feature_schemas))
  print("Query Index: ",query_index)
  query_features = long_feature_schemas[query_index]
  start_time = time.time()
  results = vptree.knn(query_features, k)
  end_time = time.time()
  times_optimized.append(end_time - start_time)
  print("Time elapsed for the query:", end_time - start_time, "seconds")

  start_time = time.time()
  results = vptree.knn_not_optimized(query_features, k)
  end_time = time.time()
  times_not_optimized.append(end_time - start_time)
  print("Time elapsed for the query:", end_time - start_time, "seconds")
for time in times_optimized:
  print(time)

print("\n")
print("\n")
print("\n")
for time in times_not_optimized:
  print(time)

In [ ]:
#TIME EVALUATION wrt bucket size
import random 

with open(GDRIVE_DIR+"features_extracted_until_gap_layer", 'rb') as features_file:
  feature_schemas = pickle.load(features_file)

MAX_LEVEL = 5



long_feature_schemas = list()
for i in range(0, 3):
    long_feature_schemas += feature_schemas

In [ ]:
#query selection
query_indexes = np.random.choice(len(long_feature_schemas), 10, replace=False) 

In [ ]:
import time
BUCKET_SIZE = 300
vptree = VPTree(long_feature_schemas, BUCKET_SIZE)
times = []
k=10
for i in range(0,10):  
  
  start_time = time.time()
  results = vptree.knn(long_feature_schemas[query_indexes[i]], k)
  end_time = time.time()
  times.append(end_time - start_time)
  print("Time elapsed for the query:", end_time - start_time, "seconds")

for time in times:
  print(time)

In [ ]:
#RANGE SEARCH PERFORMANCE
import random 
import time
with open(GDRIVE_DIR+"features_extracted_until_gap_layer", 'rb') as features_file:
  feature_schemas = pickle.load(features_file)

MAX_LEVEL = 5
BUCKET_SIZE = 300

long_feature_schemas = list()
for i in range(0, 3):
  long_feature_schemas += feature_schemas
vptree = VPTree(long_feature_schemas, BUCKET_SIZE)


times_optimized = []
times_not_optimized = []

for i in range(0,10):  
  print("Length: ",len(long_feature_schemas))
  query_index = random.randrange(len(long_feature_schemas))
  print("Query Index: ",query_index)
  query_features = long_feature_schemas[query_index]
  start_time = time.time()
  results = vptree.range_search(query_features,0.4)
  end_time = time.time()
  times_optimized.append(end_time - start_time)
  print("Time elapsed for the query:", end_time - start_time, "seconds")
  
  start_time = time.time()
  results = []
  for tmp in long_feature_schemas:
    if query_features.distance_from(tmp) < 0.4:
      results.append(tmp)
  end_time = time.time()
  times_not_optimized.append(end_time - start_time)
  print("Time elapsed for the query:", end_time - start_time, "seconds")
for time in times_optimized:
  print(time)

print("\n")
print("\n")
print("\n")
for time in times_not_optimized:
  print(time)

# Examples

## Examples of the pre-trained model

In [ ]:
#get a query image
query_path = '/content/gdrive/MyDrive/Project/david.jpg'
!mkdir query/
!mkdir query/query
!cp "{query_path}" ./query/query/

In [ ]:
#creating the pretrained model  
pretrained_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(229,229,3))
model = Sequential()
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
model.summary()
dnn = DNNExtractor(model, BATCH_SIZE, False)

#creating the index
MAX_LEVEL = 5
BUCKET_SIZE = 300
with open(GDRIVE_DIR+"features_extracted_not_fine_tuned", 'rb') as features_file:
  feature_schemas = pickle.load(features_file)

vptree = VPTree(feature_schemas, BUCKET_SIZE)


#extract features from the query image
query_feature_schema = dnn.extract_features_from_single_image("query",remove_classifier=False)

query_feature_schema.tensor= query_feature_schema.tensor[0]
print(query_feature_schema.tensor)
print("\n")
#knn queries
results = vptree.knn(query_feature_schema, 5)
    
for result in results:
    #print the image
    try:
      display_image(VALIDATION_SET +result[1].img_id, score=result[0])
    except:
      try:
        display_image(TRAINING_SET +result[1].img_id, score=result[0])
      except:
          try:
            display_image(DISTRACTOR_SET +result[1].img_id, score=result[0])
          except:
            pass
    print("\n")



## Examples of the dense-model

In [ ]:
#load the finetuned model  
model = tf.keras.models.load_model(os.path.join(GDRIVE_DIR, "fine_tuned_model"))
dnn = DNNExtractor(model, BATCH_SIZE, False)

In [ ]:
#creating the index
MAX_LEVEL = 5
BUCKET_SIZE = 300
with open(GDRIVE_DIR+"features_extracted_until_dense_layer", 'rb') as features_file:
  feature_schemas = pickle.load(features_file)

vptree = VPTree(feature_schemas, BUCKET_SIZE)



#extract features from the query image
query_feature_schema = dnn.extract_features_from_single_image("query",remove_classifier=True,layer_to_stop="last_dropout")
query_feature_schema.tensor = query_feature_schema.tensor[0]

print("\n")
#knn queries
results = vptree.knn(query_feature_schema, 5)
    
for result in results:
    #print the image
    try:
      display_image(VALIDATION_SET +result[1].img_id, score=result[0])
    except:
      try:
        display_image(TRAINING_SET +result[1].img_id, score=result[0])
      except:
          try:
            display_image(DISTRACTOR_SET +result[1].img_id, score=result[0])
          except:
            pass
    print("\n")


## Examples of the gap-model

In [ ]:
#creating the index
MAX_LEVEL = 5
BUCKET_SIZE = 300
with open(GDRIVE_DIR+"features_extracted_until_gap_layer", 'rb') as features_file:
  feature_schemas = pickle.load(features_file)

vptree = VPTree(feature_schemas, BUCKET_SIZE)

#extract features from the query image
query_feature_schema = dnn.extract_features_from_single_image("query",remove_classifier=True,layer_to_stop="gap")

query_feature_schema.tensor = query_feature_schema.tensor[0] 
print("\n")
#knn queries
results = vptree.knn(query_feature_schema, 5)
print(len(results))
for result in results:
    #print the image
    try:
      display_image(VALIDATION_SET +result[1].img_id, score=result[0])
    except:
      try:
        display_image(TRAINING_SET +result[1].img_id, score=result[0])
      except:
          try:
            display_image(DISTRACTOR_SET +result[1].img_id, score=result[0])
          except:
            pass
    print("\n")
